mmdet3d结果可视化

In [1]:
import numpy as np
import json
import cv2
import open3d as o3d

In [2]:
def get_3d_box_corners(bboxes):
    corners_3d_all = []
    for bbox in bboxes:
        x, y, z, l, w, h, yaw = bbox
        
        # 根据长宽高得到8个角点的相对坐标
        x_corners = l/2 * np.array([1, 1, -1, -1, 1, 1, -1, -1])  
        y_corners = w/2 * np.array([-1, 1, 1, -1, -1, 1, 1, -1])
        z_corners = h/2 * np.array([-1, -1, -1, -1, 1, 1, 1, 1])
        
        # 根据旋转角度旋转角点
        corners_3d = np.vstack((x_corners, y_corners, z_corners))
        c, s = np.cos(yaw), np.sin(yaw)
        R = np.array([[c, -s, 0], [s, c, 0], [0, 0, 1]])
        corners_3d = np.dot(R, corners_3d)
        
        # 加上bbox中心坐标
        x_corners, y_corners, z_corners = corners_3d[0]+x, corners_3d[1]+y, corners_3d[2]+(z/2)
        corners_3d = np.vstack((x_corners, y_corners, z_corners))
        corners_3d_all.append(corners_3d)
        
    return corners_3d_all


In [3]:
def get_bbox3d(pcd_path,corners):
    # pcd = pcd_path
    # pcd = o3d.io.read_point_cloud(pcd)
    # points = np.asarray(pcd.points)
    # mask = points[:, 0] > 0
    # pcd_nongrond_points = points[mask]
    # pcd = o3d.geometry.PointCloud()
    # pcd.points = o3d.utility.Vector3dVector(pcd_nongrond_points)

    lidar_bbox3d = []
    for box3d in corners:
        lidar_box3d = np.transpose(box3d)
        lidar_bbox3d.append(lidar_box3d)

    lidar_3dboxes = np.array(lidar_bbox3d)
    # return lidar_3dboxes,pcd
    return lidar_3dboxes

In [4]:
def draw_3D_box2point(lidar_3dboxes,points):
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(points)
    num = len(lidar_3dboxes)
    for n in range(num):
        lidar_3dbox = lidar_3dboxes[n]

        # 创建Open3D点云
        lines_box = np.array([[0, 1], [1, 2], [2, 3],[0, 3], [4, 5], [5, 6],
                                [6, 7], [4, 7], [0, 4], [1, 5], [2, 6], [3, 7]]) 
        colors = np.array([[1, 0, 0] for j in range(len(lines_box))])

        # 创建Open3D线集
        line_set = o3d.geometry.LineSet() #创建line对象
        #将八个顶点转换成o3d可以使用的数据类型
        line_set.lines = o3d.utility.Vector2iVector(lines_box) 
        line_set.colors = o3d.utility.Vector3dVector(colors)  #设置每条线段的颜色
        #把八个顶点的空间信息转换成o3d可以使用的数据类型
        line_set.points = o3d.utility.Vector3dVector(lidar_3dbox) 
        vis.add_geometry(line_set)  #将矩形框加入到窗口中

        # pcd.paint_uniform_color([1, 0, 0])
    # 可视化点云
    vis.run()
    vis.destroy_window()

In [5]:
# def draw_3D_box2point(lidar_3dboxes, points):
#     vis = o3d.visualization.Visualizer()
#     vis.create_window()
    
#     combined = points
#     vis.add_geometry(combined)
    
#     num = len(lidar_3dboxes)
#     geometries = []
#     for n in range(num):
#         lidar_3dbox = lidar_3dboxes[n]
#         # 创建Open3D点云
#         lines_box = np.array([[0, 1], [1, 2], [2, 3], [0, 3], [4, 5], [5, 6], [6, 7], [4, 7], [0, 4], [1, 5], [2, 6], [3, 7]])
#         colors = np.array([[1, 0, 0] for j in range(len(lines_box))])
#         # 创建Open3D线集
#         line_set = o3d.geometry.LineSet()
#         #创建line对象
#         #将八个顶点转换成o3d可以使用的数据类型
#         line_set.lines = o3d.utility.Vector2iVector(lines_box)
#         line_set.colors = o3d.utility.Vector3dVector(colors)
#         #设置每条线段的颜色
#         #把八个顶点的空间信息转换成o3d可以使用的数据类型
#         line_set.points = o3d.utility.Vector3dVector(lidar_3dbox)
        
#         # 将线框数据和点云数据合并到geometries列表中
#         geometries.append(combined + line_set)
        
#         vis.add_geometry(geometries[-1])
#         #将矩形框加入到窗口中

#     # pcd.paint_uniform_color([1, 0, 0])
#     # 可视化点云
#     vis.run()
    
#     # 合并geometries列表中的所有几何体
#     combined = geometries[0]
#     for geom in geometries[1:]:
#         combined += geom
    
#     # 保存包含点云和线框的几何体
#     o3d.io.write_point_cloud("output_pointcloud.pcd", combined)
    
#     vis.destroy_window()

In [6]:
def get_calib(calib_path):
    with open(calib_path,'r') as f:
        calib = f.readlines()

    P2 = np.array([float(x) for x in calib[2].strip('\n').split(' ')[1:]]).reshape(3, 4)
    K = np.array(P2[:12]).reshape(3, 4)[:3, :3]

    # 从字符串解析出从激光雷达到相机的变换矩阵
    Tr_velo_to_cam = np.array([float(x) for x in calib[5].strip('\n').split(' ')[1:]]).reshape(3, 4)
    Tr_velo_to_cam = np.insert(Tr_velo_to_cam, 3, values=[0, 0, 0, 1], axis=0)
    Tr = np.array([Tr_velo_to_cam[0,3],Tr_velo_to_cam[1,3],Tr_velo_to_cam[2,3]])

    R0_rect = np.array([float(x) for x in calib[4].strip('\n').split(' ')[1:]]).reshape(3, 3)
    # R0_rect = np.insert(R0_rect, 3, values=[0, 0, 0], axis=0)
    # R0_rect = np.insert(R0_rect, 3, values=[0, 0, 0, 1], axis=1)


    # 输出相机内参矩阵和外参矩阵
    # print("P2:")
    # print(P2)
    # print('K:')
    # print(K)
    # print('Tr_velo2cam:')
    # print(Tr_velo_to_cam)
    # print('T:')
    # print(Tr)
    # print('R0_rect:')
    # print(R0_rect)
    return P2,K,Tr_velo_to_cam,Tr,R0_rect


In [7]:
# def draw_3Dtoimg(img_path,corners,Tr_velo_to_cam,R0_rect,P2):
#     num = len(corners)
#     all_points_img = []
#     for i in range(num):
#         points_cam = np.append(corners[i].T, np.ones((8,1)), axis=1)
#         points_cam = np.dot(Tr_velo_to_cam, points_cam.T).T
#         points_cam = points_cam[:, :3]

#         points_rect = np.dot(points_cam, np.transpose(R0_rect))

#         # 投影到图像像素坐标
#         fx = P2[0,0] 
#         fy = P2[1,1]
#         cx = P2[0,2]
#         cy = P2[1,2]

#         points_img = np.zeros((8,2))
#         for i in range(8):
#             x = points_rect[i,0]
#             y = points_rect[i,1]
#             z = points_rect[i,2]
#             points_img[i,0] = x*fx/z + cx
#             points_img[i,1] = y*fy/z + cy
#             all_points_img.append(points_img)

#     # 读取图像
#     img = cv2.imread(img_path)
#     # 将投影点坐标转为整数
#     all_points_img = np.int0(all_points_img)

#     lines = np.array([[0, 1], [1, 2], [2, 3], [3, 0],  # 下面的4条线
#                     [4, 5], [5, 6], [6, 7], [7, 4],  # 上面的4条线
#                     [0, 4], [1, 5], [2, 6], [3, 7]]) # 连接上下面的4条线

#         # 在图像上绘制3D框的12条线
#     for points_img in all_points_img:
#         for start, end in lines:
#             start_x, start_y = points_img[start]
#             end_x, end_y = points_img[end]
#             img = cv2.line(img, (start_x, start_y), (end_x, end_y), (0, 0, 255), 2)

#     # 放大图像
#     scale_factor = 2  # 设置缩放因子,2表示放大两倍
#     new_width = int(img.shape[1] * scale_factor)
#     new_height = int(img.shape[0] * scale_factor)
#     resized_img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_LINEAR)
#     # 显示修改后的图像
#     cv2.imshow('3D Bounding Box', resized_img)
#     cv2.imwrite('D:\lidar\point\\000008\\3dbox2img.png',resized_img)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

#     return all_points_img




In [8]:
def draw_3Dtoimg(img_path, corners, Tr_velo_to_cam, R0_rect, P2,save_path='3dbox_img_coordinates.txt'):
    num = len(corners)
    
    # 读取图像
    img = cv2.imread(img_path)
    img_height, img_width, _ = img.shape

    # 将投影点坐标转为整数
    lines = np.array([[0, 1], [1, 2], [2, 3], [3, 0],  # 下面的4条线
                      [4, 5], [5, 6], [6, 7], [7, 4],  # 上面的4条线
                      [0, 4], [1, 5], [2, 6], [3, 7]])  # 连接上下面的4条线

    # 在图像上绘制3D框的12条线
    for i in range(num):
        points_cam = np.append(corners[i].T, np.ones((8, 1)), axis=1)
        points_cam = np.dot(Tr_velo_to_cam, points_cam.T).T
        points_cam = points_cam[:, :3]
        points_rect = np.dot(points_cam, np.transpose(R0_rect))
        # 投影到图像像素坐标
        fx = P2[0, 0]
        fy = P2[1, 1]
        cx = P2[0, 2]
        cy = P2[1, 2]
        points_img = np.zeros((8, 2))
        for j in range(8):
            x = points_rect[j, 0]
            y = points_rect[j, 1]
            z = points_rect[j, 2]
            points_img[j, 0] = x * fx / z + cx
            points_img[j, 1] = y * fy / z + cy
        
        # 将当前框的坐标归一化后存储在txt文件中
        with open(save_path, 'a') as f:
            normalized_points = points_img / np.array([img_width, img_height])
            for point in normalized_points:
                f.write(f'{point[0]:.6f} {point[1]:.6f}\n')
            f.write('\n')  # 在每个框之间添加一个空行
        
        points_img = np.int0(points_img)
        for start, end in lines:
            start_x, start_y = points_img[start]
            end_x, end_y = points_img[end]
            img = cv2.line(img, (start_x, start_y), (end_x, end_y), (0, 255, 0), 2)

    # 放大图像
    scale_factor = 2  # 设置缩放因子,2表示放大两倍
    new_width = int(img.shape[1] * scale_factor)
    new_height = int(img.shape[0] * scale_factor)
    resized_img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_LINEAR)

    # 显示修改后的图像
    cv2.imshow('3D Bounding Box', resized_img)
    cv2.imwrite('D:\\lidar\\point\\\\000008\\\\3dbox2img.png', resized_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    return points_img

In [9]:
def draw_2Dbox_img(_2dbox_path,image_path):
    image = cv2.imread(image_path)
    img_height, img_width, _ = image.shape
    with open(_2dbox_path , 'r') as f:
        _2dbox = f.readlines()

    for line in _2dbox:
        values = line.split(' ')
        cx = float(values[1]) * img_width
        cy = float(values[2]) * img_height
        w = float(values[3]) * img_width
        h = float(values[4]) * img_height

        x1 = int(cx - w / 2)
        y1 = int(cy - h / 2)
        x2 = int(cx + w / 2)
        y2 = int(cy + h / 2)
        color = (0, 255, 0)  # 绿色
        thickness = 2
        cv2.rectangle(image, (x1, y1), (x2, y2), color, thickness)

    cv2.imshow('Image with Bounding Box', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


    

In [10]:
def draw_2D_and_3D_boxes(image_path, _2dbox_path, corners, Tr_velo_to_cam, R0_rect, P2, save_path='3dbox_img_coordinates.txt'):
    # 读取图像
    image = cv2.imread(image_path)
    img_height, img_width, _ = image.shape

    # 读取2D检测框
    with open(_2dbox_path, 'r') as f:
        _2dbox = f.readlines()

    for line in _2dbox:
        values = line.split(' ')
        cx = float(values[1]) * img_width
        cy = float(values[2]) * img_height
        w = float(values[3]) * img_width
        h = float(values[4]) * img_height

        x1 = int(cx - w / 2)
        y1 = int(cy - h / 2)
        x2 = int(cx + w / 2)
        y2 = int(cy + h / 2)
        color = (0, 255, 0)  # 绿色
        thickness = 2
        cv2.rectangle(image, (x1, y1), (x2, y2), color, thickness)

    # 处理3D检测框
    num = len(corners)
    lines = np.array([[0, 1], [1, 2], [2, 3], [3, 0],  # 下面的4条线
                      [4, 5], [5, 6], [6, 7], [7, 4],  # 上面的4条线
                      [0, 4], [1, 5], [2, 6], [3, 7]])  # 连接上下面的4条线

    for i in range(num):
        points_cam = np.append(corners[i].T, np.ones((8, 1)), axis=1)
        points_cam = np.dot(Tr_velo_to_cam, points_cam.T).T
        points_cam = points_cam[:, :3]
        points_rect = np.dot(points_cam, np.transpose(R0_rect))
        
        fx = P2[0, 0]
        fy = P2[1, 1]
        cx = P2[0, 2]
        cy = P2[1, 2]
        points_img = np.zeros((8, 2))
        for j in range(8):
            x = points_rect[j, 0]
            y = points_rect[j, 1]
            z = points_rect[j, 2]
            points_img[j, 0] = x * fx / z + cx
            points_img[j, 1] = y * fy / z + cy
        
        with open(save_path, 'a') as f:
            normalized_points = points_img / np.array([img_width, img_height])
            for point in normalized_points:
                f.write(f'{point[0]:.6f} {point[1]:.6f}\n')
            f.write('\n')
        
        points_img = np.int0(points_img)
        for start, end in lines:
            start_x, start_y = points_img[start]
            end_x, end_y = points_img[end]
            image = cv2.line(image, (start_x, start_y), (end_x, end_y), (255, 255, 0), 2)

    # 放大图像
    scale_factor = 2
    new_width = int(image.shape[1] * scale_factor)
    new_height = int(image.shape[0] * scale_factor)
    resized_img = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_LINEAR)

    # 显示修改后的图像
    cv2.imshow('Image with 2D and 3D Bounding Boxes', resized_img)
    cv2.imwrite('D:\\lidar\\point\\\\000008\\\\3dbox2img.png', resized_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    return points_img



In [11]:
with open('D:\lidar\point\\000000\\000000.json' , 'r') as f:
    data = json.load(f)
# 解析JSON数据
labels_3d = []
scores_3d = []
bboxes_3d = []

for prediction in data.get('predictions', []):
    labels_3d.append(prediction.get('labels_3d', []))
    scores_3d.append(prediction.get('scores_3d', []))
    bboxes_3d.append(prediction.get('bboxes_3d', []))
img_path='D:\lidar\point\\000000\\000000.png'
calib_path = 'D:\lidar\point\\000000\\000000.txt'
# _2d_box_path = 'D:\lidar\point\\000008\save\labels\\000008.txt'
# pcd_path = 'D:\lidar\point\\000008\point_seg\pcd_nongrond_seg.pcd'

# pcd_path = 'D:\lidar\point\\000008\\nongrond.pcd'

corners=get_3d_box_corners(bboxes_3d)
# lidar_bbox3d,pcd=get_bbox3d(pcd_path,corners)
lidar_bbox3d=get_bbox3d(0,corners)
P2,K,Tr_velo_to_cam,Tr,R0_rect = get_calib(calib_path)
# draw_3D_box2point(lidar_bbox3d,pcd)
# draw_2Dbox_img(_2d_box_path,img_path)
all_points_img=draw_3Dtoimg(img_path,corners,Tr_velo_to_cam,R0_rect,P2)
# draw_2D_and_3D_boxes(img_path,_2d_box_path, corners, Tr_velo_to_cam, R0_rect, P2)
# print(all_points_img.shape)

ValueError: too many values to unpack (expected 7)

In [32]:
import numpy as np
import cv2

def project_to_image(corners, P2):
    """
    将3D检测框的角点投影到图像平面上
    :param corners: 3D检测框的8个角点坐标，形状为 (8, 3)
    :param P2: 3x4 投影矩阵
    :return: 投影到图像平面上的2D角点坐标，形状为 (8, 2)
    """
    corners = np.array(corners)  # 确保corners是一个NumPy数组
    num_corners = corners.shape[0]
    ones = np.ones((num_corners, 1))
    corners_hom = np.hstack((corners, ones))
    corners_2d_hom = np.dot(P2, corners_hom.T).T
    corners_2d = corners_2d_hom[:, :2] / corners_2d_hom[:, 2:3]
    return corners_2d

def get_2d_bounding_box(corners_2d):
    """
    根据2D角点坐标计算2D检测框
    :param corners_2d: 2D检测框的角点坐标，形状为 (8, 2)
    :return: 2D检测框的左上角和右下角坐标 (x1, y1, x2, y2)
    """
    x1 = np.min(corners_2d[:, 0])
    y1 = np.min(corners_2d[:, 1])
    x2 = np.max(corners_2d[:, 0])
    y2 = np.max(corners_2d[:, 1])
    return (x1, y1, x2, y2)

def compute_2d_iou(box1, box2):
    """
    计算两个2D检测框的IoU
    :param box1: (x1, y1, x2, y2) - 第一个检测框的左上角和右下角坐标
    :param box2: (x1, y1, x2, y2) - 第二个检测框的左上角和右下角坐标
    :return: IoU值
    """
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    iou = inter_area / float(box1_area + box2_area - inter_area)
    
    return iou

def compute_projected_2d_iou(_2dbox_path, corners, P2, img_width, img_height):
    """
    计算投影到图像上的2D检测框和3D检测框的IoU
    :param _2dbox_path: 包含2D检测框的文件路径
    :param corners: 3D检测框的角点坐标，形状为 (8, 3)
    :param P2: 3x4 投影矩阵
    :param img_width: 图像宽度
    :param img_height: 图像高度
    :return: 2D IoU值
    """
    # 投影3D检测框到图像平面上
    corners_2d = project_to_image(corners, P2)
    projected_box = get_2d_bounding_box(corners_2d)
    
    # 读取2D检测框
    with open(_2dbox_path, 'r') as f:
        _2dbox = f.readlines()

    for line in _2dbox:
        values = line.split(' ')
        cx = float(values[1]) * img_width
        cy = float(values[2]) * img_height
        w = float(values[3]) * img_width
        h = float(values[4]) * img_height

        x1 = int(cx - w / 2)
        y1 = int(cy - h / 2)
        x2 = int(cx + w / 2)
        y2 = int(cy + h / 2)
        detected_box = (x1, y1, x2, y2)
        
        iou = compute_2d_iou(projected_box, detected_box)
        print(f"IoU with 2D box {values[0]}: {iou}")

# 示例使用
# 设置路径和参数（假设这些变量已经定义）
# image_path = 'path_to_image.jpg'
# _2dbox_path = 'path_to_2dbox.txt'
# corners = [[x1, y1, z1], [x2, y2, z2], ..., [x8, y8, z8]]  # 3D检测框的角点坐标
# P2 = np.array([...])  # 投影矩阵

# 读取图像尺寸（假设你有一个图像路径）
image = cv2.imread(img_path)
img_height, img_width, _ = image.shape
print(corners)

# 调用函数
compute_projected_2d_iou(_2d_box_path, corners[0].T, P2, img_width, img_height)


[array([[ 1.62979327e+01,  1.67925763e+01,  1.32190008e+01,
         1.27243571e+01,  1.62979327e+01,  1.67925763e+01,
         1.32190008e+01,  1.27243571e+01],
       [-2.39620789e+00, -8.68302732e-01,  2.88605773e-01,
        -1.23929939e+00, -2.39620789e+00, -8.68302732e-01,
         2.88605773e-01, -1.23929939e+00],
       [-1.55883515e+00, -1.55883515e+00, -1.55883515e+00,
        -1.55883515e+00, -8.52346420e-05, -8.52346420e-05,
        -8.52346420e-05, -8.52346420e-05]]), array([[ 7.72608913,  8.15712758,  5.14966141,  4.71862297,  7.72608913,
         8.15712758,  5.14966141,  4.71862297],
       [-5.03006206, -3.63517763, -2.70582928, -4.10071371, -5.03006206,
        -3.63517763, -2.70582928, -4.10071371],
       [-1.58197528, -1.58197528, -1.58197528, -1.58197528, -0.15354079,
        -0.15354079, -0.15354079, -0.15354079]]), array([[ 6.62675888,  6.1238166 ,  9.59768647, 10.10062874,  6.62675888,
         6.1238166 ,  9.59768647, 10.10062874],
       [ 2.54833391,  1.0577